## Proceso de ETL en el dataset output_steam_games 

#### Este proceso se realiza con la finalidad de leer el dataset con el formato correcto, eliminar las columnas que no se necesitan para responder las consultas y hacer las transformaciones necesarias para optimizar el rendimiento de la API y el entrenamiento del modelo de aprendizaje automático.

### Se importan las librerías a utilizar en el proceso de ETL.

In [17]:
import pandas as pd
import json

### El archivo JSON se carga en un DataFrame utilizando la función read_json() de pandas.

In [18]:
df_games = pd.read_json('datasets_originales\output_steam_games.json', orient='records', lines=True, convert_dates=True)

### Visualización del DataFrame.

In [31]:
df_games

,publisher,genres,title,id,developer,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,year
88310,Kotoshiro,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,761140.0,Kotoshiro,1,0,0,0,1,...,0,0,1,0,0,1,0,0,0,2018
88311,"Making Fun, Inc.","Free to Play, Indie, RPG, Strategy",Ironbound,643980.0,Secret Level SRL,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,2018
88312,Poolians.com,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,670290.0,Poolians.com,0,0,0,0,1,...,0,0,1,0,1,0,0,0,0,2017
88313,彼岸领域,"Action, Adventure, Casual",弹炸人2222,767400.0,彼岸领域,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,2017
88315,Trickjump Games Ltd,"Action, Adventure, Simulation",Battle Royale Trainer,772540.0,Trickjump Games Ltd,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"Action, Adventure, Casual, Indie",Kebab it Up!,745400.0,Bidoniera Games,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,2018
120440,Ghost_RUS Games,"Casual, Indie, Simulation, Strategy",Colony On Mars,773640.0,"Nikita ""Ghost_RUS""",0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,2018
120441,Sacada,"Casual, Indie, Strategy",LOGistICAL: South Africa,733530.0,Sacada,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,2018
120442,Laush Studio,"Indie, Racing, Simulation",Russian Roads,610660.0,Laush Dmitriy Sergeevich,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,2018


### Se eliminan los valores faltantes del DataFrame. 

In [21]:
df_games = df_games.dropna() 

### Se visualizan las columnas del DataFrame para descartar las columnas que no son necesarias.

In [22]:
df_games.columns 

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

### Se eliminan columnas específicas del DataFrame con el objetivo de reducir su tamaño y mejorar su eficiencia.

In [23]:
df_games = df_games.drop(['reviews_url','url','tags', 'app_name', 'specs', 'price', 'early_access'], axis=1)

### Se visualiza el DataFrame para confirmar si muestra los resultados deseados. 
#### Nota: hasta este punto se decide mantener las columnas "publisher" y "developer" ya que ambas contienen datos similares y posiblemente alguna pueda ser el complemento de la otra.

In [24]:
df_games.head()

,publisher,genres,title,release_date,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,767400.0,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,772540.0,Trickjump Games Ltd


### Se convierte la lista de géneros a una cadena única separada por comas.

In [25]:
df_games['genres'] = df_games['genres'].apply(lambda x: ', '.join(x))  

## Proceso de creación de variables dummy para representar las variables categóricas de la columna "genres"
### Para que los algoritmos de aprendizaje automático puedan trabajar con estos datos, se requiere convertir la columna "genres" en una forma numérica.

In [26]:
genres_dummy = df_games['genres'].str.get_dummies(', ')

### Se concatenan las variables dummy con el DataFrame original.

In [27]:
df_games_dummy = pd.concat([df_games, genres_dummy], axis=1)

### Se extrae el año de la columna "release_date" y se crea la columna "year"


In [28]:
df_games = df_games_dummy 
# Reemplazar los valores no válidos por NaN / Replace invalid values ​​with NaN
df_games["release_date"] = pd.to_datetime(df_games["release_date"], errors='coerce')
# Filtrar el DataFrame para obtener los registros válidos / Filter the DataFrame to get valid records
df_filtered = df_games.dropna(subset=["release_date"])
# Convertir la columna "release_date" a tipo datetime / Convert "release_date" column to datetime type
df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])
# Rellenando los valores faltantes en las columna'release_date' con la moda / Filling missing values ​​in 'release_date' columns with mode
columns_fill_mode = ['release_date']
for col in columns_fill_mode:
    df_games[col].fillna(df_games[col].mode()[0], inplace=True)
# Modelado y eliminación de la columna 'release_date' / Modeling and removing 'release_date' column
df_games['year'] = df_games['release_date'].dt.year.astype('Int64')
df_games.drop('release_date', axis=1, inplace=True)

C:\Users\58424\AppData\Local\Temp\ipykernel_7376\638431184.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])


In [29]:
df_games.head(2)

,publisher,genres,title,id,developer,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,year
88310,Kotoshiro,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,761140.0,Kotoshiro,1,0,0,0,1,...,0,0,1,0,0,1,0,0,0,2018
88311,"Making Fun, Inc.","Free to Play, Indie, RPG, Strategy",Ironbound,643980.0,Secret Level SRL,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,2018


### El DataFrame obtenido de los diferentes procesos se exporta a un archivo CSV.

In [30]:
df_games.to_csv('output_steam_games_clean.csv') 